# Quantum State Tomography via MLE in Qiskit

In [2]:
# Importing standard Qiskit modules
from qiskit import QuantumCircuit, QuantumRegister, IBMQ, execute, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.monitor import job_monitor
from qiskit.circuit import Parameter, Instruction
from qiskit.quantum_info import Pauli

# Import state tomography modules
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.ignis.verification.tomography.fitters.lstsq_fit import lstsq_fit
from qiskit.ignis.verification.tomography.fitters.cvx_fit import cvx_fit

from qiskit.quantum_info import state_fidelity
from qiskit.opflow import Zero, One, I, X, Y, Z

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import copy
import qutip as qt

import seaborn as sns

## Target State Parities
We consider the target state $|110\rangle$ and generate the parity values associated with each Pauli string. 

In [3]:
g = qt.basis(2,0)
e = qt.basis(2,1)

# fidelity: the reconstructed state has the (flipped) ordering |q5q3q1> 
target_state_qt = qt.tensor(e,e,g)
target_state_qt = qt.ket2dm(target_state_qt)
target_state = target_state_qt.full()


# parity: "XYZ" corresponds to X measurement on q1, Y measurement on q3, and Z measurement on q5
target_state_parity_qt = qt.ket2dm(qt.tensor(g,e,e))
target_state_parity = target_state_parity_qt.full()

In [50]:
pauli = {"X":qt.sigmax(),"Y":qt.sigmay(),"Z":qt.sigmaz(),"I":qt.identity(2)}
target_parity = {}
for k1, p1 in pauli.items():
    for k2, p2 in pauli.items():
        for k3, p3 in pauli.items():
            pauli_string = k1+k2+k3
            if pauli_string == "III":
                continue
            op = qt.tensor(p1,p2,p3)
            meas = (target_state_parity_qt*op).tr()
            target_parity[pauli_string] = meas

In [51]:
target_parity

{'XXX': 0.0,
 'XXY': 0.0,
 'XXZ': 0.0,
 'XXI': 0.0,
 'XYX': 0.0,
 'XYY': 0.0,
 'XYZ': 0.0,
 'XYI': 0.0,
 'XZX': 0.0,
 'XZY': 0.0,
 'XZZ': 0.0,
 'XZI': 0.0,
 'XIX': 0.0,
 'XIY': 0.0,
 'XIZ': 0.0,
 'XII': 0.0,
 'YXX': 0.0,
 'YXY': 0.0,
 'YXZ': 0.0,
 'YXI': 0.0,
 'YYX': 0.0,
 'YYY': 0.0,
 'YYZ': 0.0,
 'YYI': 0.0,
 'YZX': 0.0,
 'YZY': 0.0,
 'YZZ': 0.0,
 'YZI': 0.0,
 'YIX': 0.0,
 'YIY': 0.0,
 'YIZ': 0.0,
 'YII': 0.0,
 'ZXX': 0.0,
 'ZXY': 0.0,
 'ZXZ': 0.0,
 'ZXI': 0.0,
 'ZYX': 0.0,
 'ZYY': 0.0,
 'ZYZ': 0.0,
 'ZYI': 0.0,
 'ZZX': 0.0,
 'ZZY': 0.0,
 'ZZZ': 1.0,
 'ZZI': -1.0,
 'ZIX': 0.0,
 'ZIY': 0.0,
 'ZIZ': -1.0,
 'ZII': 1.0,
 'IXX': 0.0,
 'IXY': 0.0,
 'IXZ': 0.0,
 'IXI': 0.0,
 'IYX': 0.0,
 'IYY': 0.0,
 'IYZ': 0.0,
 'IYI': 0.0,
 'IZX': 0.0,
 'IZY': 0.0,
 'IZZ': 1.0,
 'IZI': -1.0,
 'IIX': 0.0,
 'IIY': 0.0,
 'IIZ': -1.0}

## Tomography

In [52]:
basis_matrix = []
data = []

for pauli, val in target_parity.items():
    row = Pauli(pauli).to_matrix().flatten(order='F')
    basis_matrix.append(row)
    data.append(val)

basis_matrix = np.array(basis_matrix)

In [53]:
rho_fit = lstsq_fit(data=data, basis_matrix=basis_matrix, weights=None, trace=1)

In [54]:
rho_fit

array([[ 2.05104583e-33+0.00000000e+00j, -7.54204590e-33-3.45677104e-33j,
         8.39287368e-34-6.96645881e-33j, -1.78243901e-17+4.16333634e-17j,
         4.63048254e-33+5.82159527e-33j, -1.84805682e-33+2.26129551e-33j,
        -2.17883612e-34-6.99786862e-33j,  3.70371236e-33+4.70091180e-33j],
       [-7.54204590e-33+3.45677104e-33j,  3.35593293e-32+0.00000000e+00j,
         8.65485741e-33+2.70313689e-32j, -4.62429300e-18-1.83133681e-16j,
        -2.68386172e-32-1.36029241e-32j,  2.98450109e-33-1.14298293e-32j,
         1.25951929e-32+2.53651439e-32j, -2.15419586e-32-1.10439302e-32j],
       [ 8.39287368e-34+6.96645881e-33j,  8.65485741e-33-2.70313689e-32j,
         2.40052908e-32+0.00000000e+00j, -1.48703111e-16-4.35048900e-17j,
        -1.78784879e-32+1.81098134e-32j, -8.43680453e-33-5.35167706e-33j,
         2.36793814e-32-3.60357620e-33j, -1.44512760e-32+1.45034182e-32j],
       [-1.78243901e-17-4.16333634e-17j, -4.62429300e-18+1.83133681e-16j,
        -1.48703111e-16+4.35048900e

In [33]:
state_fidelity(rho_fit, target_state_parity)

QiskitError: 'Input quantum state is not a valid'